In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages   org.apache.spark:spark-avro_2.11:2.4.3  pyspark-shell'
#os.environ['JAVA_HOME'] = '/usr/bin/java/'

import shutil
import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark import SparkContext


In [2]:
# set the write properties for spartk context
SparkContext.setSystemProperty('spark.executor.memory', '50g')
SparkContext.setSystemProperty("spark.ui.port", "0")


sc = SparkContext("local", "orc_writer").getOrCreate()
sql_sc = SQLContext(sc)

## Input dirs

In [3]:
input_dir = '../datasets/dataset_multiformat'

In [4]:
datasets = ['nyt','tweets','beer']

In [5]:
dataset_d = {}
for dataset in datasets:
    dataset_d[dataset] = [fn for fn in os.listdir(os.path.join(input_dir,dataset)) if '.csv' in fn]

# Read and Write Files to orc

In [6]:
for dataset_dir,fn_ls in dataset_d.items():
    for fn in fn_ls:
        input_file_path = os.path.join(input_dir,dataset_dir,fn)        
        # reading input file path
        print("Reading {}".format(input_file_path))
        
        df = sql_sc.read.csv(input_file_path, header='true',inferSchema='true')
        output_dir = os.path.join(input_dir,dataset_dir)
        
        output_file_path_arvo = os.path.join(output_dir,'arvo',fn[:-4])
        shutil.rmtree(output_file_path_arvo, ignore_errors=True)
        df.write.format("avro").save(output_file_path_arvo)
        print("Completed {}".format(output_file_path_arvo))

#         output_file_path_orc = os.path.join(output_dir,'orc',fn[:-4])
#         output_file_path_json = os.path.join(output_dir,'json',fn[:-4])
#         output_file_path_parquet = os.path.join(output_dir,'parquet',fn[:-4])
        
#         shutil.rmtree(output_file_path_orc, ignore_errors=True)   
#         shutil.rmtree(output_file_path_json, ignore_errors=True)   
#         shutil.rmtree(output_file_path_parquet, ignore_errors=True)   
        
#         df.write.format("orc").save(output_file_path_orc)
#         print("Completed {}".format(output_file_path_orc))
        
#         df.write.format("json").save(output_file_path_json)
#         print("Completed {}".format(output_file_path_json))

#         df.write.format("parquet").save(output_file_path_parquet)
#         print("Completed {}".format(output_file_path_parquet))
        
        print("\n"+"--"*10+"\n")

Reading ../datasets/dataset_multiformat/nyt/CommentsJan2018.csv
Completed ../datasets/dataset_multiformat/nyt/arvo/CommentsJan2018

--------------------

Reading ../datasets/dataset_multiformat/nyt/ArticlesApril2017.csv
Completed ../datasets/dataset_multiformat/nyt/arvo/ArticlesApril2017

--------------------

Reading ../datasets/dataset_multiformat/nyt/CommentsMarch2018.csv
Completed ../datasets/dataset_multiformat/nyt/arvo/CommentsMarch2018

--------------------

Reading ../datasets/dataset_multiformat/nyt/CommentsJan2017.csv
Completed ../datasets/dataset_multiformat/nyt/arvo/CommentsJan2017

--------------------

Reading ../datasets/dataset_multiformat/nyt/ArticlesFeb2018.csv
Completed ../datasets/dataset_multiformat/nyt/arvo/ArticlesFeb2018

--------------------

Reading ../datasets/dataset_multiformat/nyt/ArticlesApril2018.csv
Completed ../datasets/dataset_multiformat/nyt/arvo/ArticlesApril2018

--------------------

Reading ../datasets/dataset_multiformat/nyt/CommentsFeb2018.csv


# Test written FIle

In [31]:
output_file = '../../dataset/nyt/orc/CommentsApril2017'
orc_df = sql_sc.read.orc(output_file)

In [34]:
orc_df

DataFrame[approveDate: string, commentBody: string, commentID: string, commentSequence: string, commentTitle: string, commentType: string, createDate: string, depth: string, editorsSelection: string, parentID: string, parentUserDisplayName: string, permID: string, picURL: string, recommendations: string, recommendedFlag: string, replyCount: string, reportAbuseFlag: string, sharing: string, status: string, timespeople: string, trusted: string, updateDate: string, userDisplayName: string, userID: string, userLocation: string, userTitle: string, userURL: string, inReplyTo: string, articleID: string, sectionName: string, newDesk: string, articleWordCount: string, printPage: string, typeOfMaterial: string]

In [16]:
df.write.format("json").save('test')